In [1]:
#Change ClsStrt from 0/1 , cls_num , dataload , Rate , MthdMgntud

In [2]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [3]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

In [4]:
rate0=235
rateNet=235                                                                  # Real rate , Load data+ '0'
cls_num=2
for i in range (1,cls_num+1):                                                  #Cls from 0to1 >>   replace:(1,cls_num+1)
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Strawberry0(i)                # Real rate    >>   Load data+'0'

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)                                                             #   4 case 1 to 0
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):                                                    #Cls from 0to1 >> (1,cls_num) 
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

first_train_shape= (370, 236)
first_test_shape= (613, 236)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [132 238]
max(train_feature_Altitude)= 3.7227
min(train_feature_Altitude)= -2.1276
first_train_sample=
 [ 1.0000e+00 -3.1047e-01 -3.0916e-01 -3.1179e-01 -2.9259e-01 -2.4866e-01
 -1.9342e-01 -1.6343e-01 -1.6633e-01 -1.9158e-01 -2.2840e-01 -2.5655e-01
 -2.7654e-01 -2.9943e-01 -3.2099e-01 -3.2704e-01 -3.0416e-01 -2.3893e-01
 -1.4160e-01 -6.9266e-02 -1.5342e-02  1.0329e-01  2.6900e-01  4.2656e-01
  6.0517e-01  7.7693e-01  8.6189e-01  8.9162e-01  9.4081e-01  1.0510e+00
  1.2486e+00  1.4782e+00  1.6436e+00  1.7783e+00  1.9924e+00  2.2715e+00
  2.4793e+00  2.5630e+00  2.5853e+00  2.6240e+00  2.7276e+00  2.9065e+00
  3.0877e+00  3.1716e+00  3.1293e+00  2.9857e+00  2.8286e+00  2.7529e+00
  2.7197e+00  2.5375e+00  2.1403e+00  1.7415e+00  1.5555e+00  1.5876e+00
  1.6757e+00  1.6555e+00  1.4953e+00  1.2717e+00  1.0700e+00  9.2266e-01
  8.2139e-01  7.5142e-01  6.9881e-01  6.5699e-01  6.3358

In [5]:
wdth=int(len(ecg1[0]))
os.chdir(cwd)
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (132, 236)
shape base_wn= (370, 236)
shape furir_windws0= (370, 235)
shape furir_windws= (370, 236)
smpl_rte= 235
mx_wndws= 238


In [6]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    #print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    #print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [7]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [8]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls60,aug_pls61,aug_pls62=2,5,3 #2 is MgWrp and 4 is GDO and 5 is freq Augmentes Data for GAN Training
    vars()['ecg_aug'+str(i)+'60']=np.append(vars()['ecg_aug'+str(i)+'60'],GAN_Gnrt(ecg,i,aug_pls60),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'60'],60,i)
    print(str(i)+'60 =',len(vars()['ecg_aug'+str(i)+'60']))
    '''vars()['ecg_aug'+str(i)+'61']=np.append(vars()['ecg_aug'+str(i)+'61'],GAN_Gnrt(ecg,i,aug_pls61),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'61'],61,i)
    print(str(i)+'61 =',len(vars()['ecg_aug'+str(i)+'61']))
    vars()['ecg_aug'+str(i)+'62']=np.append(vars()['ecg_aug'+str(i)+'62'],GAN_Gnrt(ecg,i,aug_pls62),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'62'],62,i)
    print(str(i)+'62 =',len(vars()['ecg_aug'+str(i)+'62']))'''




 Class  1  Shape =  (370, 236)
for class  1 number of windows is =  (236,)
GAN Train Data shape= (476, 235)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Optimal cluster number:  3
nise shape= (8, 90)
generated_signals shape= (8, 80, 1)


ValueError: Input 0 of layer "sequential_1" is incompatible with the layer: expected shape=(None, 235, 1), found shape=(8, 80, 1)

In [ ]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    aug_pls70,aug_pls71,aug_pls72=2,5,3 #2 is MgWrp and 3 is TimeWrp Augmentes Data for LSTM Augmentation
    vars()['ecg_aug'+str(i)+'70']=np.append(vars()['ecg_aug'+str(i)+'70'],LSTM_Gnrt_Pridc(ecg,i,aug_pls70),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'70'],70,i)
    print(str(i)+'70 =',len(vars()['ecg_aug'+str(i)+'70']))
    '''vars()['ecg_aug'+str(i)+'71']=np.append(vars()['ecg_aug'+str(i)+'71'],LSTM_Gnrt_Pridc(ecg,i,aug_pls71),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'71'],71,i)
    print(str(i)+'71 =',len(vars()['ecg_aug'+str(i)+'71']))
    vars()['ecg_aug'+str(i)+'72']=np.append(vars()['ecg_aug'+str(i)+'72'],LSTM_Gnrt_Pridc(ecg,i,aug_pls72),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'72'],72,i)
    print(str(i)+'72 =',len(vars()['ecg_aug'+str(i)+'72']))'''